In [ ]:
import polars as pl
from datetime import datetime, date, timedelta
from io import StringIO

In [ ]:
pl.__version__

In [ ]:
df = pl.DataFrame(
    {
        "foo": [1, 2, 3],
        "bar": [6.0, 7.0, 8.0],
        "ham": ["a", "b", "c"],
    }
)
print(df)

In [ ]:
other_df = pl.DataFrame(
    {
        "apple": ["x", "y", "z"],
        "ham": ["a", "b", "d"],
    }
)
print(other_df)

In [ ]:
df.join(other_df, on="ham")

In [ ]:
# df.join(other_df, on="ham", how="inner")
# df.join(other_df, on="ham", how="left")
df.join(other_df, on="ham", how="left", coalesce=False) # Don't merge join column. make a <JOIN_COL>_right col
# df.join(other_df, on="ham", how="right")
# df.join(other_df, on="ham", how="full")
# df.join(other_df, how="cross") # on not requires when how="cross"
# df.join(other_df, on="ham", how="semi")
# df.join(other_df, on="ham", how="anti")

In [ ]:
df.join(other_df, on="ham", how="full", coalesce=True) # merge join columns when how="full"

In [ ]:
props = """property_name,group
Old Ken Road,brown
Whitechapel Road,brown
The Shire,fantasy
Kings Cross Station,stations
"The Angel,Islington",light_blue
"""
props_csv = StringIO(props)
props_groups = pl.read_csv(props_csv)
print(props_groups)

prices = """property_name,cost
Old Ken Road,60
Whitechapel Road,60
Sesame Street,100
Kings Cross Station,200
"The Angel,Islington",100
"""
prices_csv = StringIO(prices)
props_prices = pl.read_csv(prices_csv)
print(props_prices)

In [ ]:
props_groups.join(props_prices, on="property_name") # Very very trivial

In [ ]:
props_groups2 = props_groups.with_columns(pl.col('property_name').str.to_lowercase().alias('property_name'))
props_groups2

In [ ]:
props_prices2 = props_prices.select(pl.col("property_name").alias("name"), pl.col("cost"))
props_prices2

In [ ]:
# how do we now apply the join between props_groups2 and props_prices2. We use left_on, and right_on where both are pl.Expr expressions
res = props_groups2.join(props_prices2, left_on=pl.col('property_name'), right_on=pl.col('name').str.to_lowercase())\
                         .drop('property_name').rename({'name': 'property_name'})\
                         .select(pl.col('property_name'), pl.exclude('property_name'))
res

In [ ]:
players = pl.DataFrame(
    {
        "name": ["Alice", "Bob"],
        "cash": [78, 135],
    }
)
print(players)
print(props_prices)

In [ ]:
# How would you join rows where a person has the money to buy the property? You can't join on any common column.
# here we can use join_where.
players.join_where(props_prices, pl.col('cash') > pl.col('cost'))

In [101]:
df_trades = pl.DataFrame(
    {
        "time": [
            datetime(2020, 1, 1, 9, 1, 0),
            datetime(2020, 1, 1, 9, 1, 0),
            datetime(2020, 1, 1, 9, 3, 0),
            datetime(2020, 1, 1, 9, 6, 0),
        ],
        "stock": ["A", "B", "B", "C"],
        "trade": [101, 299, 301, 500],
    }
)
print(df_trades)

shape: (4, 3)
┌─────────────────────┬───────┬───────┐
│ time                ┆ stock ┆ trade │
│ ---                 ┆ ---   ┆ ---   │
│ datetime[μs]        ┆ str   ┆ i64   │
╞═════════════════════╪═══════╪═══════╡
│ 2020-01-01 09:01:00 ┆ A     ┆ 101   │
│ 2020-01-01 09:01:00 ┆ B     ┆ 299   │
│ 2020-01-01 09:03:00 ┆ B     ┆ 301   │
│ 2020-01-01 09:06:00 ┆ C     ┆ 500   │
└─────────────────────┴───────┴───────┘


In [102]:
df_quotes = pl.DataFrame(
    {
        "time": [
            datetime(2020, 1, 1, 9, 0, 0),
            datetime(2020, 1, 1, 9, 2, 0),
            datetime(2020, 1, 1, 9, 4, 0),
            datetime(2020, 1, 1, 9, 6, 0),
        ],
        "stock": ["A", "B", "C", "A"],
        "quote": [100, 300, 501, 102],
    }
)

print(df_quotes)

shape: (4, 3)
┌─────────────────────┬───────┬───────┐
│ time                ┆ stock ┆ quote │
│ ---                 ┆ ---   ┆ ---   │
│ datetime[μs]        ┆ str   ┆ i64   │
╞═════════════════════╪═══════╪═══════╡
│ 2020-01-01 09:00:00 ┆ A     ┆ 100   │
│ 2020-01-01 09:02:00 ┆ B     ┆ 300   │
│ 2020-01-01 09:04:00 ┆ C     ┆ 501   │
│ 2020-01-01 09:06:00 ┆ A     ┆ 102   │
└─────────────────────┴───────┴───────┘


In [108]:
market = df_trades.join(df_quotes, on='stock', how='left')
market

time,stock,trade,time_right,quote
datetime[μs],str,i64,datetime[μs],i64
2020-01-01 09:01:00,"""A""",101,2020-01-01 09:00:00,100
2020-01-01 09:01:00,"""A""",101,2020-01-01 09:06:00,102
2020-01-01 09:01:00,"""B""",299,2020-01-01 09:02:00,300
2020-01-01 09:03:00,"""B""",301,2020-01-01 09:02:00,300
2020-01-01 09:06:00,"""C""",500,2020-01-01 09:04:00,501


In [109]:
# You want to produce a dataframe showing for each trade the most recent quote provided on or before the time of the trade.
market = market.with_columns(time_diff=pl.when((pl.col('time') - pl.col('time_right') > 0)).then(True).otherwise(False)).remove(pl.col('time_diff') == False)
market

time,stock,trade,time_right,quote,time_diff
datetime[μs],str,i64,datetime[μs],i64,bool
2020-01-01 09:01:00,"""A""",101,2020-01-01 09:00:00,100,true
2020-01-01 09:03:00,"""B""",301,2020-01-01 09:02:00,300,true
2020-01-01 09:06:00,"""C""",500,2020-01-01 09:04:00,501,true


In [105]:
market.group_by(['time', 'stock', 'trade']).agg(pl.col('time_right').max(), pl.col('quote')).with_columns(pl.col('quote').list.first().alias('quote')).sort(by='stock')

time,stock,trade,time_right,quote
datetime[μs],str,i64,datetime[μs],i64
2020-01-01 09:01:00,"""A""",101,2020-01-01 09:06:00,100
2020-01-01 09:01:00,"""B""",299,2020-01-01 09:02:00,300
2020-01-01 09:03:00,"""B""",301,2020-01-01 09:02:00,300
2020-01-01 09:06:00,"""C""",500,2020-01-01 09:04:00,501


In [100]:
# the easier way is to use join_asof.
df_asof_join = df_trades.join_asof(df_quotes, on="time", by="stock")
print(df_asof_join)

shape: (4, 4)
┌─────────────────────┬───────┬───────┬───────┐
│ time                ┆ stock ┆ trade ┆ quote │
│ ---                 ┆ ---   ┆ ---   ┆ ---   │
│ datetime[μs]        ┆ str   ┆ i64   ┆ i64   │
╞═════════════════════╪═══════╪═══════╪═══════╡
│ 2020-01-01 09:01:00 ┆ A     ┆ 101   ┆ 100   │
│ 2020-01-01 09:01:00 ┆ B     ┆ 299   ┆ null  │
│ 2020-01-01 09:03:00 ┆ B     ┆ 301   ┆ 300   │
│ 2020-01-01 09:06:00 ┆ C     ┆ 500   ┆ 501   │
└─────────────────────┴───────┴───────┴───────┘


/tmp/ipykernel_29348/3924691432.py:2: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  df_asof_join = df_trades.join_asof(df_quotes, on="time", by="stock")
